In [ ]:
import pandas as pd
import csv
import string
import numpy as np

In [ ]:
search = 'NUMBER'
url = f'https://classic.set.or.th/set/commonslookup.do?language=en&country=US&prefix={search}'

In [ ]:
# make array to scrape company list
alphabet = list(string.ascii_uppercase)
search_array = ['NUMBER']
search_array = np.concatenate((search_array,alphabet), axis=0)

#scrapes a list of company tickers, names and market
search_array
df_all = pd.DataFrame()
for idx in search_array:
    url_to_scrape_company_list = F'https://classic.set.or.th/set/commonslookup.do?language=en&country=US&prefix={idx}'
    df_all = pd.concat([df_all, pd.read_html(url_to_scrape_company_list)[0]],ignore_index=True)

#replaces "&" with "%26" so tickers with "&" can be searched by URL
i = 0
for tickers in df_all['Symbol']:
    df_all['Symbol'][i] = df_all['Symbol'][i].replace('&', '%26')
    i += 1




In [ ]:
#define function that finds the index number for the fundamental multiples table
def find_correct_index(list_of_data_frame):
    for number in range(1,7):
            if list_of_data_frame[number+14][0][0] == 'Accumulated: Turnover Ratio, Value Trade/Day':
                return number+14

In [ ]:
list_to_add = []

#for every ticker, find the correct index number and record the YTD P/E
for ticker in df_all['Symbol']:
    temp_list = pd.read_html(f'https://classic.set.or.th/set/factsheet.do?symbol={ticker}&ssoPageId=3&language=en&country=TH')
    try:
        list_to_add.append(temp_list[find_correct_index(temp_list)][1][7])
    except:
        print(ticker)

In [ ]:
#iterate through every entry and turn string into float (for excel)
i = 0
for number in list_to_add:
    try:
        list_to_add[i] = float (number)
    except:
        list_to_add[i] = list_to_add[i]
    i += 1

In [ ]:
#creates new column in df_all and exports to excel
df_all['P/E'] = list_to_add
df_all.to_excel("output.xlsx")